## AutoML using H2O
#### Tabular Playground Series - Jan 2021

#### Description:

In this notebbok we are going to use H2O's AutoML. It is one of the largest used AutoML libraries and is known for giving very good results. For the sake of demonstration I am going to try only for 3 model search but you can always experiement with it and train it for longer duration.

The following notebook has been inspired from various tutorials and kernels that have used H2O's AutoML to secure good ranks. Personally I found the results quite satisfactory after using this kernel conisdering the amount of work and time I had to spend to achieve that score.

Specially Work According to this Example [In Here](https://youtu.be/OOB5Fwpo7zQ?list=PLepFSk65ELHhLjvVIvPIwYJ3lnptNdT0R&t=635)

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

In [2]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM 18.9 (build 11.0.12+8-LTS-237, mixed mode)
  Starting server from C:\ProgramData\Anaconda3\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Asus\AppData\Local\Temp\tmpqeo9wasg
  JVM stdout: C:\Users\Asus\AppData\Local\Temp\tmpqeo9wasg\h2o_Asus_started_from_python.out
  JVM stderr: C:\Users\Asus\AppData\Local\Temp\tmpqeo9wasg\h2o_Asus_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,25 secs
H2O_cluster_timezone:,Asia/Colombo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.2
H2O_cluster_version_age:,19 days
H2O_cluster_name:,H2O_from_python_Asus_ciudm9
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.973 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


## IMPORTING DATASET

H2O has its own way of handling datasets and we will need to import them as a file rather than reading them as a csv

Data Set [Download](https://www.kaggle.com/prateekagrawal1405/automl-h20-ktps21/data)

In [3]:
train = h2o.import_file("D:\\DC Universe\\Ucsc\\Third Year\\ENH 3201 Industrial Placements\\H20 Applications\\Csv\\Automl_train.csv")
test = h2o.import_file("D:\\DC Universe\\Ucsc\\Third Year\\ENH 3201 Industrial Placements\\H20 Applications\\Csv\\Automl_test.csv")

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [4]:
train.describe()

Rows:300000
Cols:16




,id,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,target
type,int,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real
mins,1.0,-0.08226332148023098,-0.03139747284868896,0.020966867357024715,0.15276142274357513,0.2763766684547749,0.06616556355640804,-0.0976663005972248,0.2172599973404939,-0.24060419174975897,-0.08504600359700187,0.08327673657427467,0.08863482103728729,0.029950236962791588,0.16636741070307134,0.0
mean,249825.14585666676,0.5068728581831744,0.49789800448055244,0.521557270350856,0.5156828403788964,0.5020220135884826,0.526515230481892,0.4878900924194839,0.5251634024127899,0.45985740650068835,0.5205322691134051,0.4839264018841599,0.5068765631134305,0.5534416142060695,0.5037129309246297,7.905661328316827
maxs,499999.0,1.0162274167302328,0.8596967694315312,1.006954603242489,1.010402194425765,1.0342608913385214,1.0438577299007883,1.0661674751074297,1.0244272333729485,1.0041140988637949,1.1999513922566574,1.0226201415878613,1.0490254841877338,0.9778450539552797,0.8685064129198011,10.267568500800396
sigma,144476.73256229574,0.20397619377641654,0.22815945314715183,0.20077005864001524,0.23303548066745422,0.2207011812354522,0.21790897941119114,0.18109605419380245,0.2162214743217503,0.19668460399631932,0.20185419152962938,0.22008244024189108,0.21894739994721196,0.229730302487058,0.20823755996298204,0.733070830366318
zeros,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,1.0,0.6703898514390889,0.8112995057309422,0.6439683093331412,0.2917913764510022,0.28411737646993296,0.8559531758452059,0.8907004183744506,0.2855421109796029,0.5582454418515085,0.7794183626907151,0.9218320519913782,0.8667720988813201,0.8787327721946618,0.3054113450701753,7.243042589449295
1,3.0,0.3880525276975261,0.6211042271574185,0.6861020924830562,0.5011490796546958,0.6437895146086654,0.449804950718596,0.5108237501974809,0.5807482361435341,0.4183350707888616,0.4326316993235111,0.4398722862927277,0.4349705690738134,0.3699574333716138,0.3694841682508747,8.203331138256422
2,4.0,0.8349504778390991,0.2274363757909521,0.3015838588756856,0.293408406815278,0.6068394934817684,0.8291750847270303,0.5061434957864068,0.5587710129561313,0.5876031455771054,0.8233116378351174,0.5670066799940074,0.6777078828596778,0.8829380804527522,0.3030471034878757,7.776090759821726


In [5]:
# Prepare the data

y = 'target'
x = train.columns
x.remove(y)
x.remove('id')

In [6]:
# max_models can be played around with and seed as well. Greater the number of max_models greater is the time that its gonna take. The best part about it is that
# It even tries out various ensemble models.
aml = H2OAutoML(max_models = 3, seed = 1)
aml.train(x = x, y = y, training_frame = train)

AutoML progress: |
12:28:40.864: AutoML: XGBoost is not available; skipping it.
12:28:40.918: Step 'best_of_family_xgboost' not defined in provider 'StackedEnsemble': skipping it.
12:28:40.919: Step 'all_xgboost' not defined in provider 'StackedEnsemble': skipping it.

███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_4_AutoML_1_20220214_122840

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 0.38026560393646774
RMSE: 0.6166567959055246
MAE: 0.514275774952512
RMSLE: 0.0726438770828844
R^2: 0.30585592688903773
Mean Residual Deviance: 0.38026560393646774
Null degrees of freedom: 9922
Residual degrees of freedom: 9920
Null deviance: 5436.644077672281
Residual deviance: 3773.3755878615693
AIC: 18573.85117214369

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE

In [13]:
# h20 saves the models in a table format where it has the model name and the various parameters such as rmse, mse, mae and more
lb = aml.leaderboard

In [14]:
# Let's have a look at some of the rows in the table.
lb.head()

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_BestOfFamily_4_AutoML_1_20220214_122840,0.489669,0.699764,0.489669,0.58603,0.079581
StackedEnsemble_BestOfFamily_3_AutoML_1_20220214_122840,0.489828,0.699877,0.489828,0.585433,0.0795909
GBM_1_AutoML_1_20220214_122840,0.490354,0.700253,0.490354,0.585107,0.0796286
DRF_1_AutoML_1_20220214_122840,0.499144,0.706502,0.499144,0.592037,0.080333
StackedEnsemble_BestOfFamily_2_AutoML_1_20220214_122840,0.511337,0.715078,0.511337,0.604693,0.0812937
StackedEnsemble_BestOfFamily_1_AutoML_1_20220214_122840,0.511338,0.715079,0.511338,0.604693,0.0812938
GLM_1_AutoML_1_20220214_122840,0.527426,0.726241,0.527426,0.612557,0.0825579


In [15]:
# To view all the models and their scores we can use the rows function to display all of them.
lb.head(rows=lb.nrows)

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_BestOfFamily_4_AutoML_1_20220214_122840,0.489669,0.699764,0.489669,0.58603,0.079581
StackedEnsemble_BestOfFamily_3_AutoML_1_20220214_122840,0.489828,0.699877,0.489828,0.585433,0.0795909
GBM_1_AutoML_1_20220214_122840,0.490354,0.700253,0.490354,0.585107,0.0796286
DRF_1_AutoML_1_20220214_122840,0.499144,0.706502,0.499144,0.592037,0.080333
StackedEnsemble_BestOfFamily_2_AutoML_1_20220214_122840,0.511337,0.715078,0.511337,0.604693,0.0812937
StackedEnsemble_BestOfFamily_1_AutoML_1_20220214_122840,0.511338,0.715079,0.511338,0.604693,0.0812938
GLM_1_AutoML_1_20220214_122840,0.527426,0.726241,0.527426,0.612557,0.0825579


In [16]:
# choose the best model which is the first record in the table as our model.
model = aml.leader

In [17]:
# use the leader model to predict on the test dataset. Note we are using the test file imported in h2o and not the dataframe/
preds = model.predict(test)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [18]:
# convert the predicts into a list using the as_list function adn then create our final submission file.
final = h2o.as_list(preds)
final['predict']

0         8.002813
1         7.856504
2         7.981198
3         8.186579
4         8.109929
            ...   
199995    8.133507
199996    8.078163
199997    8.179134
199998    7.909408
199999    7.858966
Name: predict, Length: 200000, dtype: float64

In [21]:
test_df = pd.read_csv("D:\\DC Universe\\Ucsc\\Third Year\\ENH 3201 Industrial Placements\\H20 Applications\\Csv\\Automl_test.csv")

In [24]:
test_df.columns

Index(['id', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7',
       'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13', 'cont14'],
      dtype='object')

In [25]:
sub = pd.DataFrame()
sub['id'] = test_df['id']
sub['target'] = final['predict']
# sub.to_csv('submission.csv',index=False)

In [26]:
sub.head(10)

,id,target
0,0,8.002813
1,2,7.856504
2,6,7.981198
3,7,8.186579
4,10,8.109929
5,14,7.988252
6,16,8.039023
7,17,7.823659
8,18,7.842768
9,19,7.782403
